In [ ]:
!pip install langchain-ollama

In [31]:
!ollama --version

ollama version is 0.5.7


In [33]:
#Ensuring ollama is running
!ollama serve

Error: listen tcp 127.0.0.1:11434: bind: Only one usage of each socket address (protocol/network address/port) is normally permitted.


In [34]:
!ollama pull llama3

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest 
pulling 6a0746a1ec1a... 100% ▕████████████████▏ 4.7 GB                         
pulling 4fa551d4f938... 100% ▕████████████████▏  12 KB                         
pulling 8ab4849b038c... 100% ▕████████████████▏  254 B                         
pulling 577073ffcc6c... 100% ▕████████████████▏  110 B                         
pulling 3f8eb4da87fa... 100% ▕████████████████▏  485 B                         
verifying sha256 digest 
writing manifest 
success 


In [35]:
!ollama pull nomic-embed-text

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest 
pulling 970aa74c0a90... 100% ▕████████████████▏ 274 MB                         
pulling c71d239df917... 100% ▕████████████████▏  11 KB                         
pulling ce4a164fc046... 100% ▕████████████████▏   17 B                         
pulling 31df23ea7daa... 100% ▕████████████████▏  420 B                         
verifying sha256 digest 
writing manifest 
success 


In [2]:
rmdir /s /q data 

In [13]:
from langchain import hub
from langchain.chains import RetrievalQA
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.callbacks.manager import CallbackManager
from langchain_ollama import OllamaLLM
from langchain_ollama import OllamaEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory

In [14]:
FILEPATH    = "Giv.pdf"
LOCAL_MODEL = "llama3"
EMBEDDING   = "nomic-embed-text"

In [22]:
loader = PyPDFLoader(FILEPATH)
data = loader.load()
#The loaded text is split into manageable chunks using RecursiveCharacterTextSplitter to facilitate efficient processing.
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300, chunk_overlap=50)
all_splits = text_splitter.split_documents(data)

In [23]:
persist_directory = 'data'
#Chroma vector store is generated from the split documents, utilizing Ollama embeddings for semantic representation.

vectorstore = Chroma.from_documents(
    documents=all_splits,
    embedding=OllamaEmbeddings(model=EMBEDDING),
    persist_directory=persist_directory
)

In [24]:
#Defining Prompt Template and Memory Handling

llm = OllamaLLM(base_url="http://localhost:11434",
                                  model=LOCAL_MODEL,
                                  verbose=True,
                                  callback_manager=CallbackManager(
                                      [StreamingStdOutCallbackHandler()])
                                  )

retriever = vectorstore.as_retriever()

In [25]:
template = """ You are a knowledgeable chatbot, here to help with questions of the user. Your tone should be professional and informative.
    
    Context: {context}
    History: {history}

    User: {question}
    Chatbot:
    """
prompt = PromptTemplate(
        input_variables=["history", "context", "question"],
        template=template,
    )

memory = ConversationBufferMemory(
        memory_key="history",
        return_messages=True,
        input_key="question"
    )

In [26]:
#The RetrievalQA chain is instantiated

qa_chain = RetrievalQA.from_chain_type(
            llm=llm,
            chain_type='stuff',
            retriever=retriever,
            verbose=True,
            chain_type_kwargs={
                "verbose": True,
                "prompt": prompt,
                "memory": memory,
            }
        )


In [27]:
query = "What is the output of AC coupled?"

In [28]:
qa_chain.invoke({"query": query})



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
 You are a knowledgeable chatbot, here to help with questions of the user. Your tone should be professional and informative.
    
    Context: Output Data (AC)
Nominal AC Output Power 3000W
Max AC Apparent Power 3000VA
Max Output Current 15A
AC Nominal Voltage Range 202V - 262V
AC Grid Frequency Range
Power Factor At Rate Power 1
Power Factor 0.9 Leading...0.9 Lagging
THDI <3%
AC Connection Single Phase (multiple units can be installed for 3 Phase)
Battery Power
Nominal Power 3000W
Max Charging/Discharging Current 60A / 60A
Backup Output
Output Rated Power 3000VA
Peak Power 3600VA, for 10s
Output Voltage 230Vac ±2%, 50Hz

Output Data (AC)
Nominal AC Output Power 3000W
Max AC Apparent Power 3000VA
Max Output Current 15A
AC Nominal Voltage Range 202V - 262V
AC Grid Frequency Range
Power Factor At Rate Power 1
Power Factor 0.9 Leading...0.9 Laggin

{'query': 'What is the output of AC coupled?',
 'result': "According to the provided Output Data, the AC output power of this system is Nominal AC Output Power 3000W. This means that it can produce a maximum of 3000 Watts of alternating current (AC) power.\n\nAdditionally, the system has a Max AC Apparent Power of 3000VA, which represents the maximum apparent power that the system can handle. The Max Output Current is rated at 15A, and the AC Nominal Voltage Range is between 202V and 262V.\n\nIt's also important to note that the system operates within a specific frequency range, with an AC Grid Frequency Range of unknown specifications (as it is not explicitly stated). The Power Factor is listed as 1 at rated power, and it can operate up to ±2% from its nominal voltage output."}

In [29]:
query = "What is the general data of Hybrid Inverter?"
qa_chain.invoke({"query": query})



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
 You are a knowledgeable chatbot, here to help with questions of the user. Your tone should be professional and informative.
    
    Context: Yes / Opt 
Certiﬁcates And Approvals
TuV CE, TuV 62109-1&2, TuV VDE 0126-1-1, TuV AS4777&AS/NZS 3100, EN50438, 
SAA, TuVG98/1-4:2019, G98, G99, G100
•• •• •• ••
••
Hybrid Inverter
Backup Output
Max Power Output From Battery
Output Frequency
Output Voltage 230V
50Hz
2600W

Yes / Opt 
Certiﬁcates And Approvals
TuV CE, TuV 62109-1&2, TuV VDE 0126-1-1, TuV AS4777&AS/NZS 3100, EN50438, 
SAA, TuVG98/1-4:2019, G98, G99, G100
•• •• •• ••
••
Hybrid Inverter
Backup Output
Max Power Output From Battery
Output Frequency
Output Voltage 230V
50Hz
2600W

Battery Connection
AC Connection
Display
H4 / MC4
Screw Terminal
Screw Terminal
LED
Interfaces: WI-FI/USB/GPRS/RS485/4G
Warranty: 5 Years/10 Years
Opt/Yes/Opt/Yes/Yes


{'query': 'What is the general data of Hybrid Inverter?',
 'result': 'According to the provided information, here are the key specifications for the Hybrid Inverter:\n\n* Type: Hybrid Inverter\n* Certifications and Approvals: TÜV CE, T ÜV 62109-1&2, T ÜV VDE 0126-1-1, T ÜV AS4777&AS/NZS 3100, EN50438, SAA, T ÜV G98/1-4:2019, G98, G99, G100\n* Output:\n\t+ Backup Output: Not specified\n\t+ Max Power Output From Battery: 2600W\n\t+ Output Frequency: 50Hz\n\t+ Output Voltage: 230V\n\nPlease note that there is no specific data provided for the AC coupled output. However, I can inform you that based on historical information, the AC output power of this system is Nominal AC Output Power 3000W.\n\nWould you like me to provide more information or clarify any other aspects of the Hybrid Inverter?'}

In [30]:
query = "What's the weight of the hybrid inverter"
qa_chain.invoke({"query": query})



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
 You are a knowledgeable chatbot, here to help with questions of the user. Your tone should be professional and informative.
    
    Context: Yes / Opt 
Certiﬁcates And Approvals
TuV CE, TuV 62109-1&2, TuV VDE 0126-1-1, TuV AS4777&AS/NZS 3100, EN50438, 
SAA, TuVG98/1-4:2019, G98, G99, G100
•• •• •• ••
••
Hybrid Inverter
Backup Output
Max Power Output From Battery
Output Frequency
Output Voltage 230V
50Hz
2600W

Yes / Opt 
Certiﬁcates And Approvals
TuV CE, TuV 62109-1&2, TuV VDE 0126-1-1, TuV AS4777&AS/NZS 3100, EN50438, 
SAA, TuVG98/1-4:2019, G98, G99, G100
•• •• •• ••
••
Hybrid Inverter
Backup Output
Max Power Output From Battery
Output Frequency
Output Voltage 230V
50Hz
2600W

Battery Connection
AC Connection
Display
H4 / MC4
Screw Terminal
Screw Terminal
LED
Interfaces: WI-FI/USB/GPRS/RS485/4G
Warranty: 5 Years/10 Years
Opt/Yes/Opt/Yes/Yes


{'query': "What's the weight of the hybrid inverter",
 'result': "A great question! Unfortunately, the provided information does not mention the weight of the Hybrid Inverter. However, I can suggest contacting the manufacturer, GivEnergy (https://www.givenergy.co.uk), directly for this specific detail. They should be able to provide you with the necessary information.\n\nAdditionally, if you're interested in knowing more about the Hybrid Inverter or its specifications, I'd be happy to help!"}

In [31]:
query = "What are the dimensions of the hybrid inverter"
qa_chain.invoke({"query": query})



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
 You are a knowledgeable chatbot, here to help with questions of the user. Your tone should be professional and informative.
    
    Context: Yes / Opt 
Certiﬁcates And Approvals
TuV CE, TuV 62109-1&2, TuV VDE 0126-1-1, TuV AS4777&AS/NZS 3100, EN50438, 
SAA, TuVG98/1-4:2019, G98, G99, G100
•• •• •• ••
••
Hybrid Inverter
Backup Output
Max Power Output From Battery
Output Frequency
Output Voltage 230V
50Hz
2600W

Yes / Opt 
Certiﬁcates And Approvals
TuV CE, TuV 62109-1&2, TuV VDE 0126-1-1, TuV AS4777&AS/NZS 3100, EN50438, 
SAA, TuVG98/1-4:2019, G98, G99, G100
•• •• •• ••
••
Hybrid Inverter
Backup Output
Max Power Output From Battery
Output Frequency
Output Voltage 230V
50Hz
2600W

Battery Connection
AC Connection
Display
H4 / MC4
Screw Terminal
Screw Terminal
LED
Interfaces: WI-FI/USB/GPRS/RS485/4G
Warranty: 5 Years/10 Years
Opt/Yes/Opt/Yes/Yes


{'query': 'What are the dimensions of the hybrid inverter',
 'result': "I'm glad you asked! Unfortunately, the provided information does not mention the dimensions of the Hybrid Inverter. However, I can suggest contacting the manufacturer, GivEnergy (https://www.givenergy.co.uk), directly for this specific detail. They should be able to provide you with the necessary information.\n\nAdditionally, if you're interested in knowing more about the Hybrid Inverter or its specifications, I'd be happy to help!"}

In [32]:
query = "What is the warranty of AC coupled?"
qa_chain.invoke({"query": query})



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
 You are a knowledgeable chatbot, here to help with questions of the user. Your tone should be professional and informative.
    
    Context: Warranty:
Screw Terminal
Screw Terminal
LED
Opt / Yes / Opt / Yes / Opt
5 years standard / 10 years for E.ON customers
Certiﬁcates And Approvals
AA 4777, VDE-AR-N4105, VDE0126, G83, G98, IEC62109-1-2, IEC62040, EN61000-6-2,
EN61000-6-3, EN50438, G100
BATTERY UTILITY
LOAD
480mm260mm
290mm
AC Coupled

Warranty:
Screw Terminal
Screw Terminal
LED
Opt / Yes / Opt / Yes / Opt
5 years standard / 10 years for E.ON customers
Certiﬁcates And Approvals
AA 4777, VDE-AR-N4105, VDE0126, G83, G98, IEC62109-1-2, IEC62040, EN61000-6-2,
EN61000-6-3, EN50438, G100
BATTERY UTILITY
LOAD
480mm260mm
290mm
AC Coupled

Battery Connection
AC Connection
Display
H4 / MC4
Screw Terminal
Screw Terminal
LED
Opt / Yes / Opt / Yes / Opt

{'query': 'What is the warranty of AC coupled?',
 'result': 'According to the provided information, the warranty for the AC Coupled system is 5 years standard and 10 years for E.ON customers.'}